# $ \Delta $ $上市公司日報酬率標準差_i$ = $a_0$ + $a_1$ * (上市公司現股當沖比重平均)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

### 選取上市公司2012/01/06 - 2016/01/06的日報酬率與現股當沖比重資料

In [2]:
day_trade_data = pd.read_csv("allinformation_20120106_20160106_2.csv")

# find the sepecific time column
#tmp_arr = np.array(list(day_trade_data))
#result = np.where(tmp_arr=='2014/1/6')
#print(result[0])

# 把dataframe切割成兩部分(before 2014/01/06 和 after 2014/01/06)
day_trade_data_before = day_trade_data.iloc[:,:496] # 2012/01/06 - 2014/01/06
day_trade_data_after = day_trade_data.iloc[:,496:] # 2014/01/07 - 2016/01/06
#day_trade_data_number_and_compnames = day_trade_data.iloc[:,:2]

/anaconda3/envs/pyfinance/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (148,661) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
day_trade_data_before.head(n=5)

,number,Data Field,2012/1/6,2012/1/9,2012/1/10,2012/1/11,2012/1/12,2012/1/13,2012/1/16,2012/1/17,...,2013/12/20,2013/12/23,2013/12/24,2013/12/25,2013/12/26,2013/12/27,2013/12/30,2013/12/31,2014/1/2,2014/1/3
0,1101 台泥,日報酬率 %,-1.8258,-1.4306,3.0479,1.4085,0.5556,0.0000,-2.3481,1.8388,...,0.4400,0.4381,-0.9815,0.2203,0.3297,0.1095,1.9694,-0.7511,-1.8378,-1.8722
1,1101 台泥,現股當沖比重,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,1102 亞泥,日報酬率 %,-1.1662,0.1475,1.6200,0.0000,1.3043,0.2861,-1.1412,0.7215,...,0.9272,0.5249,0.1305,0.2608,-0.2601,0.1304,0.1302,0.3901,-1.2953,-0.9186
3,1102 亞泥,現股當沖比重,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,1103 嘉泥,日報酬率 %,-0.8475,-0.4274,0.8584,0.0000,-0.8511,0.4292,0.8547,1.6949,...,-0.9434,0.3175,0.3165,-0.3155,0.0000,-0.3165,0.3175,-0.6329,0.0000,0.6369


In [4]:
day_trade_data_after.head(n=5) 

,2014/1/6,2014/1/7,2014/1/8,2014/1/9,2014/1/10,2014/1/13,2014/1/14,2014/1/15,2014/1/16,2014/1/17,...,2015/12/23,2015/12/24,2015/12/25,2015/12/28,2015/12/29,2015/12/30,2015/12/31,2016/1/4,2016/1/5,2016/1/6
0,-2.1324,1.2615,-0.5663,-2.2779,2.0979,0.0000,0.6849,1.4739,-1.676,-0.4545,...,0.5505,2.0073,-0.8945,0.9025,-1.0733,-1.6275,0.3676,-4.2125,0.9560,-1.3258
1,1.0800,0.4100,0.9600,1.0500,1.9900,1.1200,1.4100,4.9700,2.460,1.4100,...,1.8800,7.6700,1.2000,3.5400,1.2500,2.3800,3.0900,4.4600,6.1300,3.2000
2,-0.6623,0.2667,-0.1330,-0.2663,0.0000,-0.4005,0.0000,0.8043,0.000,0.0000,...,1.8382,0.1805,0.0000,0.1802,-1.0791,-1.2727,0.9208,-4.1971,-0.9524,-0.1923
3,0.0300,0.0000,0.4200,0.4600,0.1000,0.3100,0.1200,2.2900,0.030,3.6800,...,5.3600,4.9900,0.1300,0.8400,2.8900,1.6500,2.0600,4.1600,1.9700,0.6200
4,-2.8481,1.9544,-0.6390,-0.6431,1.2945,-0.9585,0.0000,0.3226,0.000,-0.9646,...,1.6860,0.0000,-0.5181,-0.5208,-0.6283,0.6322,-0.3141,-1.2605,-0.5319,-0.8556


In [33]:
# 計算每間公司的日報酬率標準差
day_trade_data_before['company_std_before'] = day_trade_data_before.std(axis=1)
day_trade_data_after['company_std_after'] = day_trade_data_after.std(axis=1)

# 計算每間公司的現股當沖比重平均
day_trade_data_before['company_day_mean_before'] = day_trade_data_before.mean(axis=1)
day_trade_data_after['company_mean_after'] = day_trade_data_after.mean(axis=1)

x = day_trade_data_after.iloc[1::2]['company_mean_after']
x2 = day_trade_data_after.iloc[1::2]['company_mean_after']
y = day_trade_data_after.iloc[0::2]['company_std_after'] - day_trade_data_before.iloc[0::2]['company_std_before']

In [23]:
type(x)

pandas.core.series.Series

In [24]:
type(y)

pandas.core.series.Series

### $ \Delta $ $上市公司日報酬率標準差_i$ = 0.1606 -0.0015 * (上市公司現股當沖比重平均)

In [36]:
y = list(y)
#x = sm.add_constant(x)
results = sm.OLS(y,x).fit()

In [37]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     5.758
Date:                Thu, 06 Jun 2019   Prob (F-statistic):             0.0166
Time:                        00:33:19   Log-Likelihood:                -1093.8
No. Observations:                 950   AIC:                             2190.
Df Residuals:                     949   BIC:                             2194.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
company_mean_after     0.0258      0

### https://www.statsmodels.org/stable/index.html
### https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression